In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyreadstat

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from itertools import combinations
import itertools


In [2]:
csfbiomk3_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\CSF_Biomarker3\csfbiomk3_20231117.csv"
csfbiomk3 = pd.read_csv(csfbiomk3_file_path)
csfbiomk3 = pd.DataFrame(csfbiomk3)

# right_volume_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Volumetric_Analysis\rh_aparc_volume_20231117.csv"
# right_volume = pd.read_csv(right_volume_file_path)
# right_volume = pd.DataFrame(right_volume)


In [3]:
csfbiomk3.head()


,subject_id,visit,checkin_bin,exposurebin,age_decade,racecat_combined,eduyears,totyr_foot,chiiseas_pf,chiiyrs_pf,...,chiiyrs_pl,chiiseas_pg,chiiyrs_pg,c_dhpg,c_ne,c_dopa,c_da,c_dopac,c_catecholamine_flag_blood_dv,c_catecholamine_flag_diluted_dv
0,1001,1,2,1,1,5,16.0,7.0,4335.4,2167.7,...,84942.5,14535346.6,7267673.3,1810.88,201.77,933.36,0.1,308.06,0.0,0.0
1,1002,1,2,1,1,5,15.0,14.0,10363.1,5708.1,...,118350.2,16484427.0,8875813.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1003,1,2,1,1,5,18.0,12.0,6685.4,4863.9,...,121198.9,10065422.8,7633931.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1004,1,1,1,2,5,16.0,16.0,7701.2,6448.9,...,159639.5,15073952.1,12583453.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1005,1,3,0,2,5,21.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
print("Column Names:")
print(csfbiomk3.columns[2])

Column Names:
checkin_bin


In [5]:
#group them base on the value in the third column which indicates their level of playing
csfbiomk3_grouped = csfbiomk3.groupby(csfbiomk3.iloc[:, 2])

NFL_csfbiomk3_grouped = pd.DataFrame()
CP_csfbiomk3_grouped = pd.DataFrame()
HC_csfbiomk3_grouped = pd.DataFrame()


# group_name : 1, 2, 3   group_data: 
for group_name, group_data in csfbiomk3_grouped:
    if group_name == 1:
        NFL_csfbiomk3_grouped = pd.concat([NFL_csfbiomk3_grouped,group_data], ignore_index = True)
    if group_name == 2:
        CP_csfbiomk3_grouped = pd.concat([CP_csfbiomk3_grouped,group_data], ignore_index = True)
    if group_name == 3:
        HC_csfbiomk3_grouped = pd.concat([HC_csfbiomk3_grouped,group_data], ignore_index = True)
    
#print("DataFrame for NFL:")
#print(NFL_right_grouped.head())
NFL_csfbiomk3_grouped.head()


,subject_id,visit,checkin_bin,exposurebin,age_decade,racecat_combined,eduyears,totyr_foot,chiiseas_pf,chiiyrs_pf,...,chiiyrs_pl,chiiseas_pg,chiiyrs_pg,c_dhpg,c_ne,c_dopa,c_da,c_dopac,c_catecholamine_flag_blood_dv,c_catecholamine_flag_diluted_dv
0,1004,1,1,1,2,5,16.0,16.0,7701.2,6448.9,...,159639.5,15073952.1,12583453.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1008,1,1,1,2,3,15.0,22.0,8220.9,5421.2,...,179280.1,27916841.8,17511794.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1011,1,1,1,2,5,16.0,20.0,9307.0,9307.0,...,236256.2,18045112.7,18045112.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1015,1,1,1,1,3,19.0,17.0,9866.7,6173.3,...,157035.8,17608975.7,11406268.1,1663.58,75.48,966.1,5.46,242.34,0.0,0.0
4,1018,1,1,1,1,3,16.0,23.0,10635.9,7929.6,...,239294.3,33742596.2,24684465.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
index_of_chiiyrs_pg = NFL_csfbiomk3_grouped.columns.get_loc("chiiyrs_pg")
NFL_csfbiomk3_grouped.drop(columns=NFL_csfbiomk3_grouped.columns[[1] +list(range(3,index_of_chiiyrs_pg+1))], inplace = True)
CP_csfbiomk3_grouped.drop(columns=CP_csfbiomk3_grouped.columns[[1] +list(range(3,index_of_chiiyrs_pg+1))], inplace = True)
HC_csfbiomk3_grouped.drop(columns=HC_csfbiomk3_grouped.columns[[1] +list(range(3,index_of_chiiyrs_pg+1))], inplace = True)

NFL_csfbiomk3_grouped.head()


,subject_id,checkin_bin,c_dhpg,c_ne,c_dopa,c_da,c_dopac,c_catecholamine_flag_blood_dv,c_catecholamine_flag_diluted_dv
0,1004,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1008,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1011,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1015,1,1663.58,75.48,966.1,5.46,242.34,0.0,0.0
4,1018,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#combine all three classes
combined_csfbiomk3 = pd.concat([NFL_csfbiomk3_grouped, CP_csfbiomk3_grouped, HC_csfbiomk3_grouped], ignore_index=True)

combined_csfbiomk3.head()

,subject_id,checkin_bin,c_dhpg,c_ne,c_dopa,c_da,c_dopac,c_catecholamine_flag_blood_dv,c_catecholamine_flag_diluted_dv
0,1004,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1008,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1011,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1015,1,1663.58,75.48,966.1,5.46,242.34,0.0,0.0
4,1018,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Remove NaNs
print("Size before:", combined_csfbiomk3.shape)
combined_csfbiomk3.dropna(inplace=True)
print("Size after:", combined_csfbiomk3.shape)

Size before: (236, 9)
Size after: (156, 9)


In [9]:
# Separate based on the level of professionalism
X = combined_csfbiomk3.drop(columns=['subject_id','checkin_bin']) 
y = combined_csfbiomk3['checkin_bin']

In [10]:
X.head()

,c_dhpg,c_ne,c_dopa,c_da,c_dopac,c_catecholamine_flag_blood_dv,c_catecholamine_flag_diluted_dv
3,1663.58,75.48,966.10,5.46,242.34,0.0,0.0
5,1645.46,143.28,719.28,2.63,547.62,0.0,0.0
6,2329.56,361.80,996.36,18.40,459.57,0.0,0.0
8,2915.93,266.58,721.62,21.91,266.00,0.0,0.0
9,1660.90,207.08,685.77,11.41,335.39,0.0,0.0


In [11]:
# Splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Normalization
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
X_train =pd.DataFrame(X_train_scaled,columns= X.columns)
X_test = pd.DataFrame(X_test_scaled, columns= X.columns)

In [15]:
# RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# store evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average = 'weighted')
recall = recall_score(y_test, y_pred, average = 'weighted')
f1 = f1_score(y_test, y_pred, average = 'weighted')
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, f1:{f1}")

Accuracy: 0.5, Precision: 0.5763888888888888, Recall: 0.5, f1:0.4184659090909091


In [16]:
y_pred2  = model.predict(X_train_scaled)

accuracy_train = accuracy_score(y_train, y_pred2)
precision_bagging = precision_score(y_train, y_pred2, average='weighted')
recall_bagging = recall_score(y_train, y_pred2, average='weighted')
f1_bagging = f1_score(y_train, y_pred2, average='weighted')

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, f1:{f1}")
# print(f"Accuracy: {accuracy}")


Accuracy: 0.5, Precision: 0.5763888888888888, Recall: 0.5, f1:0.4184659090909091


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


## Try Bagging


In [15]:
from sklearn.ensemble import BaggingClassifier

bagging_model = BaggingClassifier(estimator=RandomForestClassifier(n_estimators=100, random_state=42),
                                  n_estimators=10, random_state=42)


bagging_model.fit(X_train, y_train)
    
y_pred_bagging = bagging_model.predict(X_test)

accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
precision_bagging = precision_score(y_test, y_pred_bagging, average='weighted')
recall_bagging = recall_score(y_test, y_pred_bagging, average='weighted')
f1_bagging = f1_score(y_test, y_pred_bagging, average='weighted')

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, f1:{f1}")

Accuracy: 0.46875, Precision: 0.5673076923076923, Recall: 0.46875, f1:0.4021802325581395


In [16]:
y_test

146    2
102    1
119    1
110    1
174    2
44     1
143    2
204    3
142    2
214    3
30     1
136    2
26     1
192    3
38     1
45     1
184    3
151    2
232    3
113    1
148    2
29     1
23     1
17     1
47     1
161    2
101    1
85     1
79     1
147    2
68     1
227    3
Name: checkin_bin, dtype: int64

## Try XGBoost

In [17]:
from xgboost import XGBClassifier


## Try XGBoost

In [18]:
X = combined_csfbiomk3.drop(columns=['subject_id','checkin_bin']) 
y = combined_csfbiomk3['checkin_bin']

In [19]:
y

3      1
5      1
6      1
8      1
9      1
      ..
231    3
232    3
233    3
234    3
235    3
Name: checkin_bin, Length: 156, dtype: int64

In [20]:
# Since for XGBoost, the y values should start at 0, you need to do some justification
y = y-1

In [21]:
y

3      0
5      0
6      0
8      0
9      0
      ..
231    2
232    2
233    2
234    2
235    2
Name: checkin_bin, Length: 156, dtype: int64

In [22]:
# Do the splitting again to make sure that this adjustemnt is applied to your new X and y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Normalization
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train =pd.DataFrame(X_train_scaled,columns= X.columns)
X_test = pd.DataFrame(X_test_scaled, columns= X.columns)

In [23]:
X_test.head()

,c_dhpg,c_ne,c_dopa,c_da,c_dopac,c_catecholamine_flag_blood_dv,c_catecholamine_flag_diluted_dv
0,0.641253,0.946755,0.002971,0.020749,1.039024,0.0,0.0
1,0.114351,0.052382,0.001241,0.002190,0.197893,0.0,0.0
2,0.241559,0.131131,0.000673,0.002054,0.117367,0.0,0.0
3,0.385711,0.099996,0.000527,0.001159,0.234794,0.0,0.0
4,0.227201,0.292102,0.001439,0.004748,0.316198,0.0,0.0


In [25]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(n_estimators=100, random_state=42)

xgb_model.fit(X_train, y_train)
    
y_pred_xgb = xgb_model.predict(X_test)

accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb, average='weighted')
recall_xgb = recall_score(y_test, y_pred_xgb, average='weighted')
f1_xgb = f1_score(y_test, y_pred_xgb, average='weighted')

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, f1:{f1}")



Accuracy: 0.46875, Precision: 0.5673076923076923, Recall: 0.46875, f1:0.4021802325581395


In [ ]:
y_pred2  = best_model.predict(X_train_scaled)

accuracy_train = accuracy_score(y_train, y_pred2)
precision_bagging = precision_score(y_train, y_pred2, average='weighted')
recall_bagging = recall_score(y_train, y_pred2, average='weighted')
f1_bagging = f1_score(y_train, y_pred2, average='weighted')

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, f1:{f1}")
# print(f"Accuracy: {accuracy}")
